In [1]:
import paddle

In [31]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, rec_model='/home/luffy/PaddleOCR/pretrain_models/ch_PP-OCRv4_rec_train/student.pdparams')  # need to run only once to download and load model into memory
img_path = '/home/luffy/final_menu/output.jpg'
result = ocr.ocr(img_path, cls=True)
#for idx in range(len(result)):
#    res = result[idx]
 #   for line in res:
#        print(line)

#from PIL import Image
result = result[0]
#image = Image.open(img_path).convert('RGB')
#boxes = [line[0] for line in result]
#txts = [line[1][0] for line in result]
##scores = [line[1][1] for line in result]
#im_show = draw_ocr(image, boxes, txts, scores, font_path='/home/luffy/final_menu/ppocr_img/fonts/chinese_cht.ttf')
#im_show = Image.fromarray(im_show)
#im_show.save('result.jpg')
result


[2024/06/17 05:21:55] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/luffy/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/luffy/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_

[[[[37.0, 17.0], [158.0, 17.0], [158.0, 41.0], [37.0, 41.0]],
  ('可口小點心', 0.8709226846694946)],
 [[[21.0, 58.0], [103.0, 58.0], [103.0, 75.0], [21.0, 75.0]],
  ('热狗12', 0.8869703412055969)],
 [[[98.0, 55.0], [197.0, 55.0], [197.0, 75.0], [98.0, 75.0]],
  ('口小肉豆30', 0.938947856426239)],
 [[[21.0, 76.0], [98.0, 76.0], [98.0, 94.0], [21.0, 94.0]],
  ('笑夏30', 0.7506663799285889)],
 [[[119.0, 75.0], [157.0, 75.0], [157.0, 94.0], [119.0, 94.0]],
  ('片', 0.9407376050949097)],
 [[[174.0, 74.0], [195.0, 74.0], [195.0, 92.0], [174.0, 92.0]],
  ('15', 0.9958171248435974)],
 [[[20.0, 96.0], [100.0, 96.0], [100.0, 114.0], [20.0, 114.0]],
  ('炸建排25', 0.8612722158432007)],
 [[[119.0, 95.0], [159.0, 95.0], [159.0, 114.0], [119.0, 114.0]],
  ('球', 0.9470120072364807)],
 [[[173.0, 94.0], [196.0, 94.0], [196.0, 111.0], [173.0, 111.0]],
  ('30', 0.9962042570114136)],
 [[[216.0, 92.0], [296.0, 88.0], [296.0, 109.0], [217.0, 112.0]],
  ('克35', 0.9904389381408691)],
 [[[19.0, 115.0], [217.0, 111.0], [218.0, 

In [33]:
import numpy as np
class BOX:
    def __init__(self,box):
        self.x = np.mean([cor[0] for cor in box] )
        self.y = np.mean([cor[1] for cor in box] )
        self.left = 0.5*(box[0][0]+box[3][0])
        self.right = 0.5*(box[1][0]+box[2][0])
        self.hi = 0.5*(box[0][1]+box[1][1])
        self.lo = 0.5*(box[2][1]+box[3][1])

In [38]:
def create_json(data):
    dict = {'餐點':data[0],'價格':data[1],'數量':0}
    return dict

In [39]:
all_inf = []
old_box = BOX([[0,0],[0,0],[0,0],[0,0]])
meal = ''
for inf in result:
    box = inf[0]
    new_box = BOX(box)
    if meal and new_box.y > old_box.lo:
        print(meal)
        meal = ''
    info = inf[1][0]
    length = len(info)
    #print(info)
    number = ''
    for i,c in enumerate(info):
        if c.isdigit():
            number += c
            #print(number)
            if i==length-1:
                d = create_json([meal,number])
                all_inf.append(d)
                #print(meal,number)
                meal = ''
        else:
            if number:
                d = create_json([meal,number])
                #print(meal,number)
                price = number
                d = create_json([meal,number])
                all_inf.append(d)
                #print(price)
                meal = ''
                number = ''
            meal += c
    old_box = new_box
all_inf

可口小點心
神味
黏据供惠）
就是爱热食
原味辣味
原味辣味
饮品/汤品
口


[{'餐點': '热狗', '價格': '12', '數量': 0},
 {'餐點': '口小肉豆', '價格': '30', '數量': 0},
 {'餐點': '笑夏', '價格': '30', '數量': 0},
 {'餐點': '片', '價格': '15', '數量': 0},
 {'餐點': '炸建排', '價格': '25', '數量': 0},
 {'餐點': '球', '價格': '30', '數量': 0},
 {'餐點': '克', '價格': '35', '數量': 0},
 {'餐點': '日式', '價格': '40', '數量': 0},
 {'餐點': '酥棒', '價格': '30', '數量': 0},
 {'餐點': '拉肉', '價格': '35', '數量': 0},
 {'餐點': '我要吃饭飯（', '價格': '10', '數量': 0},
 {'餐點': '保账', '價格': '80', '數量': 0},
 {'餐點': '打猪', '價格': '85', '數量': 0},
 {'餐點': '', '價格': '35', '數量': 0},
 {'餐點': '+蛋', '價格': '48', '數量': 0},
 {'餐點': '古早味肉燥拌', '價格': '40', '數量': 0},
 {'餐點': '古早味肉燥拌潮', '價格': '45', '數量': 0},
 {'餐點': '大利肉普', '價格': '75', '數量': 0},
 {'餐點': '消意', '價格': '60', '數量': 0},
 {'餐點': '泡菜意類', '價格': '75', '數量': 0},
 {'餐點': '', '價格': '65', '數量': 0},
 {'餐點': '泡菜岛', '價格': '75', '數量': 0},
 {'餐點': '', '價格': '65', '數量': 0},
 {'餐點': '皮蛋瘦肉粥', '價格': '65', '數量': 0},
 {'餐點': '玉米湯小', '價格': '15', '數量': 0},
 {'餐點': '大', '價格': '25', '數量': 0},
 {'餐點': '红茶小', '價格': '1015', '數量': 0},
 {'餐點': '